In [1]:
!pip install ibm_watson

     |████████████████████████████████| 368kB 7.5MB/s eta 0:00:01
     |████████�██████████████████████▍| 194kB 29.6MB/s eta 0:00:01��███████████████████████| 204kB 29.6MB/s 
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/b7/10/b4/509307276e9d558a6b06d8e69f8faf6cfbdf2d1b83abb6b8d4
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/6a/42/50/f96888116b329578304f9dda4693cef6f3e76e18272d22cb6c
Successfully built ibm-watson ibm-cloud-sdk-core


In [2]:
import os
from glob import glob
import IPython
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson import SpeechToTextV1
from ibm_watson import AssistantV2
from ibm_watson import TextToSpeechV1

In [3]:
recognition_service = SpeechToTextV1(IAMAuthenticator('jcHE4ZPz5ABJgq5jdABeY4LsuxNj_k1dxVT5jgsO1QLU'))
recognition_service.set_service_url('https://api.eu-gb.speech-to-text.watson.cloud.ibm.com/instances/6c775c52-327e-425a-8907-be2678856bfb')
SPEECH_EXTENSION = "*.webm"
SPEECH_AUDIOTYPE = "audio/webm"

def recognize_audio():
    while len(glob(SPEECH_EXTENSION)) == 0:
        pass
    filename = glob(SPEECH_EXTENSION)[0]
    audio_file = open(filename, "rb")
    os.popen("rm " + filename)
    result = recognition_service.recognize(audio=audio_file, content_type=SPEECH_AUDIOTYPE).get_result()
    return result["results"][0]["alternatives"][0]["transcript"]

In [6]:
assistant = AssistantV2(version='2019-02-28', authenticator=IAMAuthenticator('OkJZ5lFcFT2V4RJJ-hyoyukbweXKsFiVc6ZfQzhbq5w8'))
assistant.set_service_url('https://api.eu-gb.assistant.watson.cloud.ibm.com/instances/409764f3-d2b5-47bc-b720-9e7efedf5c36')
ASSISTANT_ID = "2a31d63c-af1b-4910-b96b-3dab0b9a9ce9"
session_id = assistant.create_session(assistant_id=ASSISTANT_ID).get_result()["session_id"]

def message_assistant(text):
    response = assistant.message(assistant_id=ASSISTANT_ID,
                                 session_id=session_id,
                                 input={'message_type': 'text', 'text': text}).get_result()
    return response["output"]["generic"][0]["text"]

In [8]:
synthesis_service = TextToSpeechV1(IAMAuthenticator('cJut1HW2HrjDdfOsRwAUCNmlPPnFKvRNMgxKml1gsS7z'))
synthesis_service.set_service_url('https://api.eu-gb.text-to-speech.watson.cloud.ibm.com/instances/7bbbd19d-7cf2-4d00-add4-645c05860019')

def speak_text(text):
    with open('temp.wav', 'wb') as audio_file:
        response = synthesis_service.synthesize(text, accept='audio/wav', voice="en-US_AllisonV3Voice").get_result()
        audio_file.write(response.content)
    return IPython.display.Audio("temp.wav", autoplay=True)

In [ ]:
speak_text(message_assistant(recognize_audio()))